# Bank Heist Project Submission

**Description**

Given a list of bank, their locations, the amount of money each bank holds, and the amount of time it would take to rob each bank; apply your knowledge of algorithms to make as much profit as possible!

**Design a solution that finds a list of banks that when robbed will maximize your profit, while following these constraints:**

- Total time for robbery cannot exceed **24 hours**. This includes travel time between banks + time it takes to rob the bank + time it takes to get to the helicopter escape zone

- Travel speed from one bank to another is **30 km/h**
    - Use the x and y coordinates to calculate the distance (in km) between banks
    - Use the distance and travel speed to calculate the amount of time it takes to get from one bank to another

- Solution must run under **3 minutes**

- You can start anywhere, but you have to end at the **helicopter escape zone** located at coordinates **(0,0)**

**Hints**

Most of the design paradigms you saw in class will work for this. Start with something that's easier (brute-force or greedy algorithm) and then work towards a better design once it works:

    - Divide-and-conquer
    - Brute Force
    - Greedy Algorithm
    - Dynamic Programming
    - Backtracking
    - Breadth-first & Depth-first search
Some we haven't covered:

    - Branch & Bound
    - Prune & Search
    
Because there are too many banks at each step, you will need to select only some candidates to explore.

If you find yourself using a nearest-neighbors type of approach, consider using a KD-Tree or a Ball Tree to speed it up.

There are good implementations of KD-Trees and nearest neighbours in scipy, sklearn and this [library](https://github.com/lmcinnes/pynndescent)

You can work your algorithm backwards (starting at the end and backing up to the starting point) or forwards (finding a starting point and looping until there is no time left). They will lead to different designs and results.

## Step 1: Imports

You can import the libraries you intend on using here or as you go along.

In [1]:
import pandas as pd
import math
import time

## Step 2: Load Data

The `bank_data.csv` file is located in the **data** folder. Make sure you use the correct path.

In [2]:
df = pd.read_csv('data/bank_data.csv')

## Step 3: Data Exploration

Explore the data set to get a general understanding of what you're working with.

Feel free to add additional cells, and remove the ones you don't use.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10000 non-null  int64  
 1   x_coordinate  10000 non-null  float64
 2   y_coordinate  10000 non-null  float64
 3   money         10000 non-null  int64  
 4   time (hr)     10000 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 390.8 KB


In [4]:
df.head()

,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393


## Step 4: Solution 

Your solution must provide:

- A list of bank IDs in order that you rob them
- Total amount you were able to rob
- Time it took to rob + escape
- The execution time for your code to run

**Important: Your solution must respect all the constraints mentioned in the description.**

Feel free to add additional cells, and remove the ones you don't use.

In [5]:
# Function to calculate the Euclidean distance between two points
def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to calculate travel time given a distance, assuming a speed of 30 km/h
def travel_time(dist):
    return dist / 30

# Add a new column 'profitability_ratio' which is the money each bank holds divided by the time it takes to rob it
df['profitability_ratio'] = df['money'] / df['time (hr)']

total_money = 0
total_time = 0
current_position = (0, 0)
selected_banks = []


start_time = time.time()

# Select banks as long as the total time is less than 24 hours
while total_time < 24 and not df.empty:
    # Calculate the distance from the current position to each bank and the profitability-to-distance ratio
    df['distance'] = df.apply(lambda row: distance(current_position[0], current_position[1], row['x_coordinate'], row['y_coordinate']), axis=1)
    df['profitability_distance_ratio'] = df['profitability_ratio'] / df['distance']
    
    # Select the most profitable bank (among the top 90) based on the profitability to distance ratio
    df_top = df.nlargest(90, 'profitability_ratio').nlargest(1, 'profitability_distance_ratio')

    # Choose the selected bank and calculate the time to travel to this bank
    selected_bank = df_top.iloc[0]
    time_to_bank = travel_time(selected_bank['distance'])
    # Calculate the total time including traveling to and robbing the selected bank
    potential_total_time = total_time + time_to_bank + selected_bank['time (hr)']

    # Check if robbing this bank keeps the total time within 24 hours
    if potential_total_time + travel_time(selected_bank['distance']) <= 24:
        # Add the money from the bank to the total, update the total time and current position
        total_money += selected_bank['money']
        total_time = potential_total_time
        current_position = (selected_bank['x_coordinate'], selected_bank['y_coordinate'])
        
        # Insert the bank ID at the beginning of the list
        selected_banks.insert(0, int(selected_bank['id']))

        # Remove the robbed bank from the DataFrame
        df = df[df['id'] != selected_bank['id']]
    else:
        # If adding this bank exceeds the time limit, break the loop
        break

selected_banks.append("Exit Point (0, 0)")

end_time = time.time()

elapsed_time = end_time - start_time
print("Elapsed time (in seconds):", elapsed_time)


print("Total money:", total_money)
print("Total time:", total_time)
print("Total banks robbed:", len(selected_banks))
print("Banks robbed:", selected_banks)

Elapsed time (in seconds): 13.653534889221191
Total money: 13775300.0
Total time: 23.874985779049272
Total banks robbed: 265
Banks robbed: [1318, 7769, 3773, 3237, 8887, 6566, 2444, 5381, 1723, 1926, 5200, 2590, 5892, 4492, 2859, 9469, 9275, 5327, 6852, 8949, 2162, 488, 2226, 4617, 6934, 3007, 3044, 2463, 4494, 1684, 1081, 6266, 4629, 3585, 232, 9448, 9713, 6339, 2442, 2626, 9640, 5166, 7665, 8846, 8231, 4807, 3582, 5572, 4794, 6535, 8859, 2190, 6764, 5836, 5298, 3665, 507, 6022, 7744, 8125, 1976, 2185, 6876, 9130, 6169, 279, 6623, 8784, 5474, 3239, 3216, 6591, 1053, 3484, 1424, 781, 8870, 3919, 2344, 1844, 1860, 8503, 1961, 4036, 4056, 4291, 757, 9170, 5184, 6478, 2458, 4610, 5631, 2, 7801, 5627, 839, 3971, 1966, 3340, 4359, 5971, 2639, 7494, 1455, 6254, 557, 359, 754, 6375, 7907, 5440, 4364, 7772, 5944, 2472, 5725, 9880, 5563, 8288, 4742, 4983, 1058, 8703, 2643, 8829, 4362, 4234, 3186, 6216, 9529, 7877, 7689, 1599, 9908, 1193, 8849, 3026, 8407, 1447, 7265, 8908, 7531, 5356, 3025, 429

### Briefly explain your solution/approach, and how it works.

I sorted my data with a ratio of the profitability and the distance to the next bank. I chose the 90 top banks as it seemed to be the optimal number without breaking my code. I started at the escape point and worked my way backwards, adding each new bank to the beginning of the list.

## Step 5: Summary and Results

1. What is the total amount of money you were able to collect?


In [6]:
total_money

13775300.0

2. What are the IDs of the banks you robbed, in order? Use the list from your solution.

In [7]:
print(selected_banks)

[1318, 7769, 3773, 3237, 8887, 6566, 2444, 5381, 1723, 1926, 5200, 2590, 5892, 4492, 2859, 9469, 9275, 5327, 6852, 8949, 2162, 488, 2226, 4617, 6934, 3007, 3044, 2463, 4494, 1684, 1081, 6266, 4629, 3585, 232, 9448, 9713, 6339, 2442, 2626, 9640, 5166, 7665, 8846, 8231, 4807, 3582, 5572, 4794, 6535, 8859, 2190, 6764, 5836, 5298, 3665, 507, 6022, 7744, 8125, 1976, 2185, 6876, 9130, 6169, 279, 6623, 8784, 5474, 3239, 3216, 6591, 1053, 3484, 1424, 781, 8870, 3919, 2344, 1844, 1860, 8503, 1961, 4036, 4056, 4291, 757, 9170, 5184, 6478, 2458, 4610, 5631, 2, 7801, 5627, 839, 3971, 1966, 3340, 4359, 5971, 2639, 7494, 1455, 6254, 557, 359, 754, 6375, 7907, 5440, 4364, 7772, 5944, 2472, 5725, 9880, 5563, 8288, 4742, 4983, 1058, 8703, 2643, 8829, 4362, 4234, 3186, 6216, 9529, 7877, 7689, 1599, 9908, 1193, 8849, 3026, 8407, 1447, 7265, 8908, 7531, 5356, 3025, 4293, 6104, 2827, 4287, 4605, 2194, 9928, 8169, 6281, 7087, 4906, 1397, 6156, 5296, 5126, 9049, 2741, 2346, 4627, 4465, 5155, 9736, 8206, 2656

3. How much time did it take to rob all the banks + escape? Does your solution meet the 24 hour constraint? 

In [8]:
total_time

23.874985779049272

4. What is the execution time of your solution? Does your solution run in 3 minutes or less?

In [9]:
elapsed_time

13.653534889221191